# Dense Cluster Extruction
データ全体をクラスタに分類するのではなく、密度の高い塊を見つけて解説する

In [1]:
# Talk to the Cityの outputs/{dataset} にある出力を読む
# このipynbはscatter/pipeline/にあることを想定している

dataset = "aipubcom"

In [2]:
import pandas as pd
import numpy as np

arguments_df = pd.read_csv(f"outputs/{dataset}/args.csv")
arguments_array = arguments_df["argument"].values

embeddings_df = pd.read_pickle(f"outputs/{dataset}/embeddings.pkl")
embeddings_array = np.asarray(embeddings_df["embedding"].values.tolist())

In [3]:
embeddings_array.shape

(5957, 1536)

In [4]:

from hdbscan import HDBSCAN
from collections import Counter

hdb = HDBSCAN(min_cluster_size=5, max_cluster_size=30, min_samples=2)
hdb.fit(embeddings_array)
counts = Counter(hdb.labels_)

print("Num dense clusters", len(counts) - 1)


Num dense clusters 48


In [6]:
# クラスタラベルを取得
cluster_labels = hdb.labels_

# クラスタ情報を含めたDataFrameを作成
result = pd.DataFrame({
    "arg-id": arguments_df["arg-id"],
    "comment-id": arguments_df["comment-id"],
    "argument": arguments_df["argument"],
    "cluster-id": cluster_labels
})
# ノイズクラスタ（-1）を除外
result_filtered = result[result["cluster-id"] != -1]

# CSVファイルに書き出し
result_filtered.to_csv("clustered_arguments.csv", index=False)

In [7]:
from tqdm import tqdm
from services.llm import request_to_chat_openai

clusters = result_filtered
cluster_ids = clusters["cluster-id"].unique()
results = []
prompt = """あなたはKJ法が得意なデータ分析者です。userのinputはグループに集まったラベルです。なぜそのラベルが一つのグループであるか解説して、それから表札をつけてください。
またその表札の興味深さを100点満点で出力せよ。ありきたりなものを0点、新しい気づきのあったものを100点とする。
出力はJSONとせよ。

# human
選挙よりも能登の復興やワクチン被害者の救済が重要だと考える意見があり、これらの問題に対する政治家の対応の少なさに不満を感じている声がある。また、神谷宗幣のような人物に対する懸念も示されている。
現政権に対して不満を持つ人々がいるという意見があり、特に能登半島の住民にとっては現状が悪夢のように感じられるのではないかという声がある。また、選挙が誰のために行われているのか疑問を持つ意見もある。
衆議院議員選挙の候補者たちが能登の復興について触れていないことに不満の声が上がっている。政治とカネの問題に焦点が当てられ、新鮮さが欠けていると感じる意見がある。
与党自民党に対し、能登半島の復興や非課税世帯の給付金などの対応が進んでいないことに不満の声が上がっており、国民の苦しい状況を考慮して選挙よりも責任を持って対応してほしいという意見がある。
選挙時に感情的な演説をする政治家たちに対して、普段の行動とのギャップに失笑する意見がある。また、能登半島の声にどれほど応えてきたのか疑問を持つ声もある。

# ai
{
    "解説": "これらの意見は、能登半島の復興やワクチン被害者の救済が選挙よりも重要であると考える人々の不満を表しています。特に、現政権や与党自民党に対する不満が強調されており、政治家の対応の少なさや選挙活動のパフォーマンスに対する批判が見られます。また、地域の問題解決を優先すべきだという意見や、特定の政党に対する不信感も示されています。",
    "表札": "能登復興優先と現政権への不満"
    "興味深さ": 80
}
"""
import json

for cluster_id in tqdm(list(sorted(cluster_ids))):
    cluster = clusters[clusters["cluster-id"] == cluster_id]
    input = "\n".join(cluster["argument"].values)
    print(cluster_id)
    print(input)
    messages = [{"role": "system", "content": prompt}, {"role": "user", "content": input}]
    response = request_to_chat_openai(messages=messages, model="gpt-4o", is_json=True)
    print(response)
    print()
    response = response.replace("```json", "").replace("```", "")

    results.append((response, input))



  0%|          | 0/48 [00:00<?, ?it/s]

0
This situation could worsen the well-being of creators and negatively impact the production of high-quality content.
The current situation risks exploiting creators who have made considerable efforts for the benefit of society, unjustly diminishing the future benefits and social status of creators.
The potential future problems include the decline and disappearance of creators and market value, as well as the risk of powerful companies crushing creators' works and profits.
The current situation allows for infringement of creators' rights and may lead to industry decline
The current situation poses clear disadvantages for human creators.
It may become challenging for existing creators to sustain their work and for newcomers to enter the industry.


  2%|▏         | 1/48 [00:04<03:51,  4.92s/it]

{
    "解説": "これらの意見は、クリエイターの権利や地位が現在の状況によって脅かされていることへの懸念を表しています。特に、クリエイターの努力が正当に評価されず、将来的な利益や社会的地位が不当に低下するリスクが指摘されています。また、強力な企業によるクリエイターの作品や利益の圧迫、業界の衰退、クリエイターの市場価値の低下と消失の可能性が懸念されています。これにより、既存のクリエイターが活動を続けることや新規参入者が業界に入ることが困難になる可能性が示唆されています。",
    "表札": "クリエイターの権利と地位の危機",
    "興味深さ": 85
}

1
Failing to do so may discourage creators from exploring new forms of expression.
It may also diminish the creative drive of artists and hinder the advancement of the arts.
If such actions continue, it may discourage human creators from producing art.
This practice may lead to a decrease in opportunities for traditional artists who put effort into their craft.
Such practices can discourage artists and hinder future creativity, potentially stifling artistic development.


  4%|▍         | 2/48 [00:10<04:07,  5.38s/it]

...

{
    "解説": "これらの意見は、AI生成コンテンツの不正使用や犯罪行為を防ぐために、適切な規制と管理が必要であるという共通の懸念を表しています。特に、無許可の使用を防ぐための厳格な規制の必要性が強調されています。AI技術の進化に伴い、その悪用を防ぐための法的枠組みの整備が求められていることが示されています。",
    "表札": "AIコンテンツの不正使用防止規制の必要性",
    "興味深さ": 70
}

46
There is a need for national regulations on AI to address issues of unauthorized content generation and protect creators.
There is a need for swift legal regulations to protect individuals from misuse of AI-generated content.
There is a need for regulations to address the unauthorized use and distribution of AI-generated content to protect creators' rights and individuals' privacy.
Clear guidelines are needed to determine when the use of AI-generated content falls outside copyright restrictions.
There is a need to regulate the use of AI-generated content to prevent infringement of intellectual property rights and ethical concerns.
There is a need for clear guidelines on the use of AI-generated content to avoid confusion and potential copyright infringement.
There is a need

 98%|█████████▊| 47/48 [03:25<00:03,  3.19s/it]

{
    "解説": "これらの意見は、AIによるコンテンツ生成に関する法的規制の必要性を強調しています。特に、無許可のコンテンツ生成や配布がクリエイターの権利や個人のプライバシーを侵害する可能性があることに対する懸念が示されています。また、知的財産権の侵害や倫理的な問題を防ぐために、AI生成コンテンツの使用に関する明確なガイドラインが求められています。これらの意見は、AI技術の進化に伴う新たな法的課題に対処するための規制の必要性を訴えています。",
    "表札": "AIコンテンツ規制の必要性と権利保護",
    "興味深さ": 85
}

47
There should be regulations and penalties in place to protect creators from unauthorized use of their work by AI.
Regulations are needed to protect creators in the use of AI technology
Regulation is necessary to protect creators and prevent the misuse of AI technology.
There should be regulations in place to prevent the unauthorized use of images in AI-generated content.
There is a need for regulations to protect creators from having their work misused by AI technologies.
There should be regulations to prevent the unauthorized use of others' work in AI-generated content.
There is a need to regulate the use of AI to protect creators and prevent unethical behavior.
There should be regulations to prevent the una

100%|██████████| 48/48 [03:29<00:00,  4.36s/it]

{
    "解説": "このグループの意見は、AI技術の不正使用からクリエイターを保護するための規制の必要性を強調しています。特に、AI生成コンテンツにおける無断使用や商業利用を防ぐための規制が求められており、クリエイターの権利を守ることが重要視されています。これらの意見は、AI技術の進化に伴う倫理的な問題やクリエイターの知的財産権の保護に対する懸念を反映しています。",
    "表札": "AI技術の不正使用からクリエイターを守る規制の必要性",
    "興味深さ": 85
}



In [59]:
# ファイル出力
with open("output.md", "w") as fo:
    for i, [result, input] in enumerate(results):
        result = json.loads(result)
        title = result["表札"]
        desc = result["解説"]
        score = result["興味深さ"]

        items = "\n".join(f"- {line}" for line in input.splitlines())
        print(f"## C{i}: {title}", file=fo)
        print(f"{desc}\n", file=fo)
        print(f"{items}\n", file=fo)


In [9]:
# 興味深い順表示
buf = []
for i, [result, input] in enumerate(results):
    result = json.loads(result)
    title = result["表札"]
    desc = result["解説"]
    score = result["興味深さ"]
    buf.append((score, i, title, desc, len(input.splitlines())))

buf.sort(reverse=True)
for score, i, title, desc, n in buf:
    print(f"{score}点(N={n}): C{i} {title}\n{desc}\n")

90点(N=10): C16 AIトレーニングデータの倫理的懸念と違法コンテンツのリスク
これらの意見は、AIモデルのトレーニングに使用されるデータセットに違法なコンテンツ、特に児童ポルノが含まれていることに対する倫理的懸念を表しています。違法にアップロードされたコンテンツから学習することは、著作権侵害を助長する可能性があり、AIが違法な素材を区別できず、犯罪活動に利用されるリスクを伴います。特に、児童ポルノの存在は倫理的かつ深刻な問題であり、無視できないとされています。これらの懸念は、AI技術の公共利用における倫理的課題を浮き彫りにしています。

85点(N=18): C47 AI技術の不正使用からクリエイターを守る規制の必要性
このグループの意見は、AI技術の不正使用からクリエイターを保護するための規制の必要性を強調しています。特に、AI生成コンテンツにおける無断使用や商業利用を防ぐための規制が求められており、クリエイターの権利を守ることが重要視されています。これらの意見は、AI技術の進化に伴う倫理的な問題やクリエイターの知的財産権の保護に対する懸念を反映しています。

85点(N=10): C46 AIコンテンツ規制の必要性と権利保護
これらの意見は、AIによるコンテンツ生成に関する法的規制の必要性を強調しています。特に、無許可のコンテンツ生成や配布がクリエイターの権利や個人のプライバシーを侵害する可能性があることに対する懸念が示されています。また、知的財産権の侵害や倫理的な問題を防ぐために、AI生成コンテンツの使用に関する明確なガイドラインが求められています。これらの意見は、AI技術の進化に伴う新たな法的課題に対処するための規制の必要性を訴えています。

85点(N=9): C42 AI時代のクリエイター権利保護の必要性
これらの意見は、AI生成コンテンツが人間のアーティストやクリエイターの権利や努力を脅かす可能性があることに対する懸念を表しています。特に、AIによって生成されたコンテンツが、クリエイターのオリジナリティや評判を損なう可能性があるため、権利保護の必要性が強調されています。これに対して、厳格な措置を講じるべきだという意見が多く見られます。全体として、AI技術の進化に伴うクリエイターの権利保護の重要性が強調されています。

85点(N=7): C